In [13]:
import torch

In [14]:
import NanoGPTLangugageModel as nano
device = nano.device

In [15]:
model = nano.NanoGPTLanguageModel()
model.load_state_dict(torch.load('model_weights.pth'))
m = model.to(nano.device)
encode, decode = nano.encode, nano.decode

In [16]:
start_str = "The"
idx = torch.tensor(encode(start_str), dtype=torch.long, device=device).unsqueeze(0)
print(decode(m.generate(idx = idx, max_new_tokens=500)[0].tolist()))

TypeError: NanoGPTLanguageModel.generate() got an unexpected keyword argument 'use_cache'